## Projet de contrôle qualité - Étude de la pollution à `Vesoul`  

## Auteur : KHELID Lilya-Nada  

### Contexte  

Ayant souffert d’asthme à Vesoul, ce projet vise à comprendre si la particule `PM2.5` en est une cause possible. Nous allons analyser ses niveaux sur les deux dernières années et évaluer si la qualité de l’air s’est dégradée ou améliorée.  

### Objectifs  

- Estimer la moyenne et la variance des niveaux de PM2.5  
- Comparer ces valeurs sur deux années différentes  
- Appliquer des tests statistiques pour analyser les variations  
- Vérifier si PM2.5 suit une loi de probabilité  
- Élaborer un protocole de contrôle qualité et voir si les niveaux restent sous contrôle  

### Méthodologie  

Nous utiliserons des outils statistiques pour :  

- Estimer les paramètres clés (moyenne, variance, médiane, quantiles)  
- Tester les différences entre les deux années (Wilcoxon, test du signe, Cox-Stuart)  
- Vérifier l’adéquation des données à une loi de probabilité  
- Analyser la corrélation entre les deux années  

### Plan  

1. Récupération et exploration des données  
2. Estimation des paramètres statistiques  
3. Comparaison des années par tests statistiques  
4. Validation d’un modèle de contrôle qualité  
5. Interprétation et recommandations  

#### Données : [Atmo France](https://www.atmo-france.org/article/les-portails-regionaux-open-data-des-aasqa)  


### 📥 Importation des librairies

In [15]:
import os 
import pandas as pd
import plotly.express as px
import scipy.stats as stats
import numpy as np
import scipy.stats as stats
import plotly.figure_factory as ff # d'autre sont possible
import plotly.express as px

### 📥 Importation de la data



Notre dataset contient les informations pour plusieurs villes et différents types de polluants. 

Nous avons appliqué un filtre afin de ne conserver que les données relatives à la ville de **Vesoul** et au polluant **PM2.5**. 

Après ce filtrage, de nombreuses variables sont devenues inutiles. Nous avons donc conservé uniquement les colonnes suivantes :
- **Date de début**
- **Date de fin**
- **Valeur**

L'unité de la **Valeur** du polluant **PM2.5** est exprimée en **µg/m³**.


In [16]:
df = pd.read_csv("../data/processed/Vesoul_PM2.5_2022_2024.csv") 
df['date_fin'] = pd.to_datetime(df['date_fin']).dt.date
df['date_debut'] = pd.to_datetime(df['date_debut'])
print(df.shape)
df.head()

(673, 3)


,date_debut,date_fin,valeur
0,2023-01-13,2023-01-13,1.8
1,2023-01-14,2023-01-14,2.0
2,2023-01-15,2023-01-15,1.9
3,2023-01-16,2023-01-16,1.7
4,2023-01-17,2023-01-17,3.3


In [17]:
print(sum(df['date_debut'] == df['date_fin']))
print(df.shape[0])

673
673


On peut retirer `date_fin`, car elle est exactement identique à `date_debut`.

In [18]:
df = df.drop('date_fin', axis=1)
df = df.rename(columns={'date_debut': 'date'})
df.head()

,date,valeur
0,2023-01-13,1.8
1,2023-01-14,2.0
2,2023-01-15,1.9
3,2023-01-16,1.7
4,2023-01-17,3.3


In [19]:
df["annee"] = df["date"].dt.year

stats_per_year = df.groupby("annee")["valeur"].agg(["mean", "var"]).rename(columns={"mean": "Moyenne", "var": "Variance"})
stats_per_year 

,Moyenne,Variance
annee,,
2023,7.388141,39.615325
2024,6.973130,28.690359


### 📊 Visualisation des données

In [20]:
fig1 = px.line(df, x="date", y="valeur", 
               title="Évolution des PM2.5 à Vesoul",
               labels={"date_debut": "Date", "valeur": "Concentration PM2.5 (µg/m³)"},
               markers=True)
fig1.show()

fig2 = px.box(df, x="annee", y="valeur",
              title="Distribution des PM2.5 par année",
              labels={"annee": "Année", "valeur": "Concentration PM2.5 (µg/m³)"})
fig2.show()

/Users/lilya/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



D'une années à l'autre, c'est trés similaire


## Tests Statistiques :

In [21]:
df_2023 = df[df["annee"] == 2023]["valeur"]
df_2024 = df[df["annee"] == 2024]["valeur"]

### 1️⃣ Test de Normalité - **Shapiro-Wilk**
📌 **Objectif :** Vérifier si les données suivent une loi normale.

- **Hypothèse nulle ($H_0$)** : Les données sont normalement distribuées.
- **Hypothèse alternative ($H_1$)** : Les données ne sont pas normalement distribuées.
- **Interprétation :** Si $p$-valeur > 0.05, on **accepte $H_0$** → Données normales.
  Sinon, on **rejette $H_0$** → Données non normales.


In [22]:
shapiro_2023 = stats.shapiro(df_2023)
shapiro_2024 = stats.shapiro(df_2024)

print(f"Test de Shapiro-Wilk (Normalité) - 2023 : p-valeur = {shapiro_2023.pvalue:.4f}")
print(f"Test de Shapiro-Wilk (Normalité) - 2024 : p-valeur = {shapiro_2024.pvalue:.4f}")

Test de Shapiro-Wilk (Normalité) - 2023 : p-valeur = 0.0000
Test de Shapiro-Wilk (Normalité) - 2024 : p-valeur = 0.0000


> Les données ne sont pas normales, on va donc utiliser des tests non parametriques

### 2️⃣ Test de Comparaison des Moyennes

#### **🔹 Test de Wilcoxon-Mann-Whitney** (non parametrique)
📌 **Objectif :** Comparer les distributions sans supposer de normalité.

- **Hypothèse nulle ($H_0$)** : Pas de différence entre les distributions de 2023 et 2024.
- **Hypothèse alternative ($H_1$)** : Une année a des valeurs significativement différentes.
- **Interprétation :** Si $p$-valeur < 0.05, on rejette $H_0$ → Différence significative.

In [23]:
mann_whitney = stats.mannwhitneyu(df_2023, df_2024, alternative="two-sided")
print(f"Test de Mann-Whitney U : p-valeur = {mann_whitney.pvalue:.4f}")

Test de Mann-Whitney U : p-valeur = 0.4628


> Pas de différence significative de distribution entre 2023 et 2024

### 3️⃣ Test de Comparaison des Variances
#### **🔹 Test de Levene**
📌 **Objectif :** Vérifier si les variances des PM2.5 sont homogènes entre les années.

- **Hypothèse nulle ($H_0$)** : Les variances des deux années sont égales.
- **Hypothèse alternative ($H_1$)** : Les variances sont différentes.
- **Interprétation :** Si $p$-valeur < 0.05, on rejette $H_0$ → Variances différentes.

In [24]:
levene = stats.levene(df_2023, df_2024)
print(f"Test de Levene (homogénéité des variances) : p-valeur = {levene.pvalue:.4f}")

Test de Levene (homogénéité des variances) : p-valeur = 0.4501


> Pas de différence significative dans la variabilité des PM2.5.

## Modélisation des PM2.5

In [25]:
pm25 = df['valeur']

In [26]:
fig = px.histogram(x=pm25, nbins=30, histnorm='probability density', title="Histogramme des PM2.5",
                   labels={'x': "Concentration PM2.5 (µg/m³)", 'y': "Densité de probabilité"}, marginal='box')
fig.show()

In [27]:
x = np.linspace(min(pm25), max(pm25), 100)

#--Loi Log-Normale--
shape, loc, scale = stats.lognorm.fit(pm25, floc=0)
pdf_lognorm = stats.lognorm.pdf(x, shape, loc, scale)

#--Loi de Weibull--
shape_weibull, loc_weibull, scale_weibull = stats.weibull_min.fit(pm25, floc=0)
pdf_weibull = stats.weibull_min.pdf(x, shape_weibull, loc_weibull, scale_weibull)
#--Loi Gamma--
shape_gamma, loc_gamma, scale_gamma = stats.gamma.fit(pm25, floc=0)
pdf_gamma = stats.gamma.pdf(x, shape_gamma, loc_gamma, scale_gamma)

fig = ff.create_distplot([pm25], ["Données Observées"], show_hist=True)
fig.add_scatter(x=x, y=pdf_lognorm, mode='lines', name='Log-Normale')
fig.add_scatter(x=x, y=pdf_weibull, mode='lines', name='Weibull')
fig.add_scatter(x=x, y=pdf_gamma, mode='lines', name='Gamma')
fig.update_layout(title="Comparaison des distributions ajustées", xaxis_title="PM2.5", yaxis_title="Densité")
fig.show()

In [32]:
# Test de Kolmogorov-Smirnov
ks_lognorm = stats.kstest(pm25, 'lognorm', args=(shape, loc, scale))
ks_gamma = stats.kstest(pm25, 'gamma', args=(shape_gamma, loc_gamma, scale_gamma))
ks_weibull = stats.kstest(pm25, 'weibull_min', args=(shape_weibull, loc_weibull, scale_weibull))

ad_weibull = stats.anderson(pm25, dist='weibull_min') # Valide si la loi est adapté sur les extremes (queue de distribution)

print(f"Test de Kolmogorov-Smirnov - Log-Normale : p-valeur = {ks_lognorm.pvalue:.4f}")
print(f"Test de Kolmogorov-Smirnov - Gamma : p-valeur = {ks_gamma.pvalue:.4f}")
print(f"Test de Kolmogorov-Smirnov - Weibull : p-valeur = {ks_weibull.pvalue:.4f}")

print("\nTest d'Anderson-Darling :")
print(f"Weibull : statistique = {ad_weibull.statistic:.4f}, seuils critiques = {ad_weibull.critical_values}, p-valeurs = {ad_weibull.significance_level}")

Test de Kolmogorov-Smirnov - Log-Normale : p-valeur = 0.0170
Test de Kolmogorov-Smirnov - Gamma : p-valeur = 0.0001
Test de Kolmogorov-Smirnov - Weibull : p-valeur = 0.0000

Test d'Anderson-Darling :
Weibull : statistique = 14.9321, seuils critiques = [0.342 0.472 0.563 0.636 0.758 0.88  1.043 1.168], p-valeurs = [0.5   0.75  0.85  0.9   0.95  0.975 0.99  0.995]


Visuellement, on a des lois qui semblent se raprocher pas mal de nos données mais les tests stats ne le valide pas. 

#### Choix de loi :

La distribution **log-normale** est choisie car, bien qu’elle ne soit pas *statistiquement validée*, elle épouse visuellement la forme des données observées, capturant mieux la dissymétrie (sur la gauche) et l’étalement des PM2.5 que les distributions Gamma et Weibull.

## Carte de controle 

> Nous utilisons les données de 2023 comme année de référence pour fixer un seuil et analyser l’évolution de la pollution en 2024.

Définition du seuil basé sur 2023

In [33]:
# Trouve les parametres de la loi log normal sur 2023
shape_ln, loc_ln, scale_ln = stats.lognorm.fit(df_2023)

# Seuil à 90% 
seuil_90 = stats.lognorm.ppf(0.90, shape_ln, loc_ln, scale_ln)
print(f"Seuil de pollution à 90% : {seuil_90:.2f} µg/m³")

Seuil de pollution à 90% : 13.54 µg/m³


Vérification des dépassements en 2024

In [35]:
# Nombre de depassement en 2024 au dessus de 90% par rapport a 2023
nb_depassements = np.sum(df_2024 > seuil_90)
pourcentage_depassement = (nb_depassements / len(df_2024)) * 100

print(f"Seuil de pollution basé sur 2023 : {seuil_90:.2f} µg/m³")
print(f"Nombre de jours où la pollution dépasse le seuil en 2024 : {nb_depassements}")
print(f"Pourcentage de dépassement en 2024 : {pourcentage_depassement:.2f}%")

if pourcentage_depassement > 10: # si cela arrive plus de 10% (arbitraire) du temps on a : 
    print("🚨 La pollution en 2024 dépasse souvent le seuil : QUALITÉ D'AIR DÉGRADÉE")
else:
    print("✅ La pollution en 2024 reste sous contrôle : QUALITÉ D'AIR STABLE")

Seuil de pollution basé sur 2023 : 13.54 µg/m³
Nombre de jours où la pollution dépasse le seuil en 2024 : 33
Pourcentage de dépassement en 2024 : 9.14%
✅ La pollution en 2024 reste sous contrôle : QUALITÉ D'AIR STABLE


Carte de contrôle (Shewhart) basée sur 2023 pour surveiller 2024

In [39]:
fig = px.line(df[df["annee"] == 2024], x="date", y="valeur",
              title="Carte de Contrôle des PM2.5 en 2024",
              labels={"date": "Date", "valeur": "Concentration PM2.5 (µg/m³)"},
              markers=True)

# 2023 comme ligne de référence
fig.add_hline(y=np.mean(df_2023), line_dash="dash", line_color="blue", annotation_text="Moyenne 2023 (Référence)")

# seuil de pollution basé sur 2023 (quantile 90%)
fig.add_hline(y=seuil_90, line_dash="dot", line_color="red", annotation_text="Seuil 90%")

fig.show()

/Users/lilya/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



CUSUM pour détecter des dérives progressives

In [45]:
# référence 2023
mu_2023 = np.mean(df_2023)  
sigma_2023 = np.std(df_2023)  
k = sigma_2023 / 2  # Tolérance
h = 5 * sigma_2023  # Seuil d'alerte


df_2024_copy = df[df["annee"] == 2024].copy()  
df_2024_copy["cusum"] = 0  
for i in range(1, len(df_2024)):
    df_2024_copy.loc[df_2024_copy.index[i], "cusum"] = max(
        0, df_2024_copy.loc[df_2024_copy.index[i - 1], "cusum"] + (df_2024_copy.loc[df_2024_copy.index[i], "valeur"] - mu_2023 - k)
    )


fig = px.line(df_2024_copy, x="date", y="cusum", title="Carte de Contrôle CUSUM des PM2.5 en 2024",
              labels={"date": "Date", "cusum": "CUSUM (Somme des écarts cumulés)"},
              markers=True)
fig.add_hline(y=h, line_dash="dash", line_color="red", annotation_text="Seuil d'alerte")

fig.show()

/var/folders/db/swcwxrj54c30_jn72llv15kh0000gn/T/ipykernel_93448/1920035504.py:11: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '7.469871030617355' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/lilya/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



📌 Interprétation du CUSUM

📊 Si la courbe CUSUM reste proche de 0 → Pas de dérive majeure ✅

📊 Si elle monte progressivement → Hausse lente de la pollution 🚨

📊 Si elle dépasse le seuil  h  → Alerte pollution élevée et durable ❌

> De **janvier à mars 2024**, une forte hausse du CUSUM indique une pollution persistante, suivie d’une amélioration jusqu’à octobre. Cependant, **fin 2024 - début 2025**, le dépassement du seuil d’alerte signale un risque élevé.



## 📊 Résultats

### 1️⃣ Comparaison 2023 vs 2024
- **Pas de variation significative** des niveaux de pollution (tests statistiques).
- **Stabilité de la variabilité**, aucune tendance alarmante détectée.

✅ **Conclusion** : La pollution est globalement inchangée.

### 2️⃣ Modélisation et Seuil Critique
- **Loi log-normale** retenue pour modéliser les PM2.5.
- **Seuil 90%** défini sur 2023 pour identifier les dépassements en 2024.

✅ **Conclusion** : Seuil fixé pour mesurer les écarts.

### 3️⃣ Contrôle Qualité & Impact Santé
- **9.14% des jours de 2024 dépassent le seuil critique**, mais restent dans une marge acceptable.
- **Carte de contrôle Shewhart** : Des pics ponctuels de pollution sont visibles, notamment en début et fin d’année, mais aucun dépassement critique prolongé.
- **CUSUM** : Une forte hausse en début d’année suivie d’un retour à la normale, avec une légère reprise en fin d’année, mais aucune tendance alarmante sur la période globale.

✅ **Conclusion** : La pollution PM2.5 en 2024 reste sous contrôle malgré des fluctuations saisonnières.


📌 **Synthèse** : **La qualité de l’air en 2024 reste globalement stable et correct par rapport à 2023, mais des périodes de forte pollution en début et fin d’année nécessitent une vigilance accrue pour prévenir les impacts sur la santé, notamment chez les personnes asthmatiques.**

